In [25]:
import os
import requests
import zipfile
import glob
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
url_basica = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte = "microdatos/enigh"
fin = "_ns_viviendas_csv.zip"
# Lista de años (ejemplo)
years = [2020, 2022]

In [3]:
# Crear carpeta 'datos' si no existe
os.makedirs("datos", exist_ok=True)

In [ ]:
# Ciclo para descargar los archivos por año
for year in years:
    url = f"{url_basica}{year}/{segunda_parte}{year}{fin}"
    destfile = f"datos/{year}{fin}"
# Descargar el archivo
    response = requests.get(url, stream=True)
    with open(destfile, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)

    # Descomprimir el archivo
    with zipfile.ZipFile(destfile, 'r') as zip_ref:
        zip_ref.extractall("datos")

    # Renombrar el archivo
    os.rename("datos/viviendas.csv", f"datos/viviendas_{year}.csv")

    print(f"Procesado: {year}")

In [5]:
df = pd.read_csv(os.path.join("datos", "viviendas_2020.csv"))
df.describe()

C:\Users\COMIMSA\AppData\Local\Temp\ipykernel_16008\1334521193.py:1: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join("datos", "viviendas_2020.csv"))


,folioviv,mat_pared,mat_techos,cocina,cuart_dorm,num_cuarto,disp_agua,excusado,drenaje,disp_elect,combustible,eli_basura,tenencia,lavadero,fregadero,regadera,tinaco_azo,cisterna,pileta,calent_sol,calent_gas,medidor_luz,bomba_agua,tanque_gas,aire_acond,calefacc,tot_resid,tot_hom,tot_muj,tot_hog,ubica_geo,tam_loc,est_socio,est_dis,upm,factor,procaptar
count,8.775400e+04,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.00000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000,87754.000000
mean,1.607354e+09,7.777890,8.396723,1.068840,2.047656,3.733505,1.58623,1.019566,1.495658,1.036933,2.770916,1.562755,3.331461,1.177143,1.385099,1.247556,1.431502,1.819530,1.631208,1.889920,1.654511,1.070219,1.754256,1.909691,1.789434,1.968697,3.598047,1.749960,1.848087,1.014267,15859.606707,2.521834,2.072441,259.666807,5168.394352,401.518130,0.003624
std,9.284120e+08,0.725728,2.760956,0.253184,0.897583,1.493519,1.42921,0.138504,0.946839,0.346952,0.869538,1.157859,1.142057,0.381792,0.486622,0.431595,0.495289,0.384581,0.482480,0.312992,0.475530,0.255517,0.430530,0.286626,0.407713,0.174137,1.866857,1.161537,1.212966,0.132909,9296.094764,1.316022,0.814935,161.828361,2911.969178,406.892039,0.060089
min,1.000136e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1001.000000,1.000000,1.000000,1.000000,1.000000,10.000000,0.000000
25%,8.042402e+08,8.000000,7.000000,1.000000,1.000000,3.000000,1.00000,1.000000,1.000000,1.000000,3.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,8019.000000,1.000000,2.000000,119.000000,2706.000000,146.000000,0.000000
50%,1.561158e+09,8.000000,10.000000,1.000000,2.000000,4.000000,1.00000,1.000000,1.000000,1.000000,3.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,1.000000,15106.000000,3.000000,2.000000,253.000000,5155.000000,279.000000,0.000000
75%,2.402962e+09,8.000000,10.000000,1.000000,3.000000,5.000000,2.00000,1.000000,2.000000,1.000000,3.000000,1.000000,4.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,5.000000,2.000000,3.000000,1.000000,24024.000000,4.000000,2.000000,388.000000,7686.000000,501.000000,0.000000
max,3.260771e+09,8.000000,10.000000,2.000000,13.000000,23.000000,7.00000,2.000000,5.000000,5.000000,6.000000,8.000000,6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,25.000000,13.000000,12.000000,5.000000,32057.000000,4.000000,4.000000,558.000000,10149.000000,5284.000000,1.000000


In [54]:
# Definimos función de estimación
def estimacion(path, variable):
    # Cargar los datos
    datos = pd.read_csv(path, low_memory=False)    
    datos['cve_ent'] = datos['folioviv'].astype(str).apply(lambda x: '0' + x[0] if len(x) == 9 else x[:2]) # Obtenermos clave geoestadistica de acuerdo columna
    datos = datos[datos['cve_ent'] == '09'] # Filtramos por entidad de interes
    cols_mat = [col for col in datos.columns if col.startswith('mat')] # Reemplazo de valores no numéricos en columnas que inician con "mat"
    for col in cols_mat:
        datos[col] = datos[col].astype(str).replace(['', 'NA', '&'], '0') # Remplaza cadenas de texto por 0
        datos[col] = pd.to_numeric(datos[col], errors='coerce').fillna(0) # Convierte los valores a numéricos y asegura que no queden NaN
    # Diseño muestral usando statsmodels y columna
    datos['factor'] = pd.to_numeric(datos['factor'], errors='coerce').fillna(0)
    weights = datos['factor']
    print("Sacar stats")
    mydesign = DescrStatsW(datos, weights=weights)
    print(f"Fin mydesign {mydesign.sum}")
    
    # Estimar viviendas con disponibilidad de agua diaria
    print(datos.dotac_agua.dtypes)
    datos['dotac_agua'] = pd.to_numeric(datos['dotac_agua'], errors='coerce').fillna(0).astype('Int64')
    print(datos.dotac_agua.dtypes)
    agua_disponible = datos.groupby('dotac_agua').apply(
        lambda x: pd.Series({
            'viviendas': mydesign.sum,
            'porcentaje': mydesign.mean * 100
        })
    ).reset_index()
    # Filtrar viviendas con agua diaria (dotac_agua == 1)
    agua_disponible = agua_disponible[agua_disponible['dotac_agua'] == 1]
    agua_disponible['year'] = path[15:19]  # Extraer el año desde la ruta del archivo
    if variable == "dispagua":
        return agua_disponible
    else:
        raise ValueError("Variable no reconocida")

In [55]:
# Obtener la lista de archivos CSV en la carpeta "data"
file_paths = glob.glob("datos/viviendas_*.csv")


# Aplicar la función de estimación a cada archivo y combinar los resultados
tabla_disp_agua = pd.concat(
    [estimacion(file, "dispagua") for file in file_paths], ignore_index=True
)

print("Porcentaje de viviendas con disponibilidad de agua diaria")
print(tabla_disp_agua)

TypeError: can't multiply sequence by non-int of type 'float'

In [26]:
# Ejemplo de validación
data = np.array([1, 2, 3, 4, 5])
weights = np.array([1, 2, 1, 3, 2])


print(data.dtype)  # Asegúrate de que sea numérico (por ejemplo, float64 o int64)
print(weights.dtype)  # Lo mismo aplica aquí

int32
int32


In [27]:
mydesign = DescrStatsW(data, weights=weights)

In [31]:
mydesign